In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [22]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY Year
  ORDER BY Year
  """).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    296800.75|
|2011|     302141.9|
|2012|    298233.42|
|2013|    299999.39|
|2014|    299073.89|
|2015|    307908.86|
|2016|    296050.24|
|2017|    296576.69|
+----+-------------+



In [24]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY Year
  ORDER BY Year
  """).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [43]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  SELECT date_built AS Year, ROUND(AVG(price),2) AS Average_Price, COUNT(*)
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY Year
  ORDER BY Year
  """).show()


+----+-------------+--------+
|Year|Average_Price|count(1)|
+----+-------------+--------+
|2010|    285010.22|     176|
|2011|    276553.81|     171|
|2012|    307539.97|     154|
|2013|    303676.79|     160|
|2014|    298264.72|     174|
|2015|    297609.97|     187|
|2016|     293965.1|     172|
|2017|    280317.58|     182|
+----+-------------+--------+



In [42]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
  SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price, COUNT(*) AS Num_Homes
  FROM home_sales
  GROUP BY View_Rating
  HAVING AVG(price) >= 350000
  ORDER BY View_Rating DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.4312269687652588 seconds ---


In [44]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [45]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [48]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
  SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price, COUNT(*) AS Num_Homes
  FROM home_sales
  GROUP BY View_Rating
  HAVING AVG(price) >= 350000
  ORDER BY View_Rating DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.37000036239624023 seconds ---


### NOTE:
Slight speed improvement with caching- .43 seconds to .37 seconds 

In [51]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [53]:
# 11. Read the formatted parquet data.
partitioned_df=spark.read.parquet('home_sales_partitioned')

In [54]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView('home_sales_p')

In [64]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql("""
  SELECT view AS View_Rating, ROUND(AVG(price),2) AS Average_Price, COUNT(*) AS Num_Homes
  FROM home_sales_p
  GROUP BY View_Rating
  HAVING AVG(price) >= 350000
  ORDER BY View_Rating DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.6492888927459717 seconds ---


### NOTE:
Result- .65 seconds </br>
Slower when partiioned by date_built, but query is not grouped by date_built. 

In [65]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [66]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False